In [2]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
sns.set_style("whitegrid")
from pprint import pprint
import sqlite3
import yaml
import re
from datetime import datetime
import maya
from collections import Counter
from glob import glob
import ast
import json
from sklearn.preprocessing import StandardScaler

try: # for pip >= 10
    from pip._internal.req import parse_requirements
except ImportError: # for pip <= 9.0.3
    from pip.req import parse_requirements

from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

import networkx as nx
import networkx.algorithms.community as nxcom

import bokeh.io
from bokeh.io import output_file, show
from bokeh.resources import INLINE
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,
                         BoxZoomTool, ResetTool, OpenURL, CustomJS, Column, SaveTool)
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_notebook
from bokeh.models.graphs import from_networkx
from bokeh.models import TextInput, Button

%matplotlib inline

# Code for hiding seaborn warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_json("binder-clean.json", orient="index").reset_index()
pip_df = df[df["pip"].apply(lambda x: len(x) != 0)]
pip_df

,index,channels,conda,pip,apt
0,ab1be186a74c81c83f863bc3b6258fee46eeca56,[],[],"[numpy==1.18.5, pandas==1.0.5, fuzzywuzzy==0.1...",[]
3,6464ab1b75787590096275bf36b6e3035f23d2ab,[],[],"[requests==2.24.0, numpy==1.19.2, matplotlib==...",[]
4,f4d256f2e4e8688261871073f9e7f87cacd7486c,[],[],"[requests==2.24.0, numpy==1.19.2, matplotlib==...",[]
5,562d55fbc439fbf936f32fcb9e09fbfa1f145936,[],[],"[requests==2.24.0, beautifulsoup4==4.9.3, ufit...",[]
6,770ff3fac88cd8e4b49876a551514d3c3028740e,[],[],"[requests==2.24.0, beautifulsoup4==4.9.3, ufit...",[]
...,...,...,...,...,...
84618,c4d2c75d1807a1d1189b84bd6f4a0aafca5b8c53,[],[],"[numpy, scipy, scikit-learn, matplotlib, gensi...",[]
84619,959514fb6451184747ee0d26dc359fd87c1d6447,[],[],"[backcall==0.1.0, bleach==2.1.4, cycler==0.10....",[]
84621,24698dfd180e2bb0de1805a8ce2040ba914d70df,[],[],"[bs4, requests]",[]
84632,3e4c9af0dfdd2be3ebbd7f71ad9f463db9b915d4,[],[],"[scipy, numpy, matplotlib]",[]


In [10]:
dep_table = pd.DataFrame()
temp_df = pd.DataFrame()
index = 0
for recipe in pip_df["pip"]:
    clean_recipe = [i.split('=', 1)[0].split('<')[0].split('>')[0].split('[')[0].split('~')[0].split(';')[0].strip()
                                      .lower() for i in recipe]
    for i in clean_recipe:
        temp_df.at[index, i] = True
    index += 1
    if (index % 1000 == 0) or index == len(pip_df):
        print(index)
        dep_table = dep_table.append(temp_df)
        temp_df = pd.DataFrame()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
34420


In [11]:
dep_table

,numpy,pandas,fuzzywuzzy,bs4,geopandas,pycountry,matplotlib,seaborn,bokeh,sklearn,...,instapy,instapy-chromedriver,optimade-client,guessit,imdbparser,imdb-cli-tool,pyforest,hpbandster,pyswip,gputil
0,True,True,True,True,True,True,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34415,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34416,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34417,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34418,True,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
dep_table = dep_table.fillna(0)

In [8]:
dep_table.to_hdf('dependency_table.h5', key='df', mode='w')